In [ ]:
from datasets import load_dataset
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from transformers import DistilBertForSequenceClassification, file_utils, Pipeline, AutoTokenizer, pipeline
import matplotlib.pyplot as plt
import seaborn as sns
from convokit import Corpus, download

In [ ]:
model_path = "./results/models/best"
model_describ = "distilbert-base-cased"

In [ ]:
# load best model
model2 = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_describ)

In [ ]:
labels = [
    "admiration",
    "amusement",
    "anger",
    "annoyance",
    "approval",
    "caring",
    "confusion",
    "curiosity",
    "desire",
    "disappointment",
    "disapproval",
    "disgust",
    "embarrassment",
    "excitement",
    "fear",
    "gratitude",
    "grief",
    "joy",
    "love",
    "nervousness",
    "optimism",
    "pride",
    "realization",
    "relief",
    "remorse",
    "sadness",
    "surprise",
    "neutral"
  ]
id2label = {i:label for i,label in enumerate(labels)}

In [ ]:
# This is pretty much the source code from TextClassificationPipeline, but when i subclassed it
# didnt correctly work(not sure why, didn't try again at the end) so i just copied the whole code.
# We cant use TextClassification itself, since it only outputs the highest label
if file_utils.is_tf_available():
    from transformers.models.auto.modeling_tf_auto import TF_MODEL_FOR_SEQUENCE_CLASSIFICATION_MAPPING

if file_utils.is_torch_available():
    from transformers.models.auto.modeling_auto import MODEL_FOR_SEQUENCE_CLASSIFICATION_MAPPING

class MultiLabelTextClassification(Pipeline):
    """
    Text classification pipeline using any :obj:`ModelForSequenceClassification`. See the `sequence classification
    examples <../task_summary.html#sequence-classification>`__ for more information.

    This text classification pipeline can currently be loaded from :func:`~transformers.pipeline` using the following
    task identifier: :obj:`"sentiment-analysis"` (for classifying sequences according to positive or negative
    sentiments).

    If multiple classification labels are available (:obj:`model.config.num_labels >= 2`), the pipeline will run a
    softmax over the results. If there is a single label, the pipeline will run a sigmoid over the result.

    The models that this pipeline can use are models that have been fine-tuned on a sequence classification task. See
    the up-to-date list of available models on `huggingface.co/models
    <https://huggingface.co/models?filter=text-classification>`__.
    """

    def __init__(self, return_all_scores: bool = False, **kwargs):
        super().__init__(**kwargs)

        self.check_model_type(
            TF_MODEL_FOR_SEQUENCE_CLASSIFICATION_MAPPING
            if self.framework == "tf"
            else MODEL_FOR_SEQUENCE_CLASSIFICATION_MAPPING
        )

        self.return_all_scores = return_all_scores


    def __call__(self, *args, **kwargs):
        """
        Classify the text(s) given as inputs.

        Args:
            args (:obj:`str` or :obj:`List[str]`):
                One or several texts (or one list of prompts) to classify.

        Return:
            A list or a list of list of :obj:`dict`: Each result comes as list of dictionaries with the following keys:

            - **label** (:obj:`str`) -- The label predicted.
            - **score** (:obj:`float`) -- The corresponding probability.

            If ``self.return_all_scores=True``, one such dictionary is returned per label.
        """
        outputs = super().__call__(*args, **kwargs)


        scores = np.exp(outputs) / (1+np.exp(outputs))
        if self.return_all_scores:
            return [
                [{"label": self.model.config.id2label[i], "score": score.item()} for i, score in enumerate(item)]
                for item in scores
            ]
        else:
            return [
                {"label": self.model.config.id2label[item.argmax()], "score": item.max().item()} for item in scores
            ]

def analyze_result(result, threshold = 0.5):
    """Sort the results and throw away all labels with prediction under threshold"""
    output = []
    for sample in result:
        sample = np.array(sample)
        scores = np.array([label['score'] for label in sample])
        predicted_samples = np.argwhere(scores > threshold).reshape(-1)
        output.append(sorted(sample[predicted_samples], key = lambda item: item['score'], reverse=True))
    return output

In [ ]:
pipeline_config = {
    "return_all_scores":True,
    "device":0    
}
inference_pipeline = MultiLabelTextClassification(model=model2, tokenizer=tokenizer, **pipeline_config)

In [ ]:
zero_shot_classifier = "typeform/distilbert-base-uncased-mnli"

In [ ]:
temp = pipeline("zero-shot-classification", device=0, model=zero_shot_classifier, tokenizer = zero_shot_classifier)

# Read Dataset

In [ ]:
dataset = pd.read_csv("data/friends-final-raw.txt", sep="\t")

In [ ]:
corpus = Corpus(filename=download("tennis-corpus"))
utterances = list(corpus.iter_utterances())
texts = [t.text for t in utterances]
speakers = [t.get_speaker().id for t in utterances]
dataset = pd.DataFrame(zip(texts, speakers), columns=['line', 'person']) # todo: change name

In [ ]:
# split into sentences
dataset["line"] = dataset["line"].str.split(r'[.!?]+\s')
person_counts = dataset["person"].value_counts()
main_characters = person_counts[person_counts > 1000].reset_index()["index"]
print(main_characters)

dataset = dataset[dataset["person"].isin(main_characters)]

In [ ]:
predicted_emotions = {}
for p in main_characters:
    predicted_emotions[p] = list()

for row in tqdm(dataset.itertuples(), total=len(dataset)):
    prediction = temp(row.line, labels, multi_label=True)
    if len(row.line) == 1:
        prediction = [prediction]
    #x = inference_pipeline(row.line)
    prediction = [[{'label' : label, 'score': value} for label, value in zip(sentence['labels'], sentence['scores'])] for sentence in prediction]
    result = analyze_result(prediction, .8)
    result = [(pred['label'], pred['score']) for pred in result[0]] 
    predicted_emotions[row.person].append(result)

In [ ]:
total_emotions_per_person = {}
for p in main_characters:
    total_emotions_per_person[p] = {}
    for l in labels:
        total_emotions_per_person[p][l] = 0

for person, sentences in predicted_emotions.items():    
    for s in sentences:
        for e in s:
            total_emotions_per_person[person][e[0]] += (e[1]/ len(sentences))

# Plotting

In [ ]:
for person, emotions in total_emotions_per_person.items():
    plt.title(person)
    plt.ylim((0,0.1))
    plt.xticks(rotation='vertical')
    plt.bar(range(0, len(labels)), emotions.values(), tick_label=labels)
    plt.show()

In [ ]:
df = pd.DataFrame(columns=["person", "emotion", "value"])

for person, emotions in total_emotions_per_person.items():
    for emotion, value in emotions.items():
        df.loc[len(df)] = [person,emotion,value]
        
df = df[df["emotion"] != "neutral"]
sns.set(rc={'figure.figsize':(15,8)})
sns.set_theme(style="whitegrid")
chart = sns.barplot(x="emotion", y="value", hue="person", data=df)
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)